In [ ]:
# ! pip install edsl

In [ ]:
# import os
# os.environ['OPENAI_API_KEY'] = 

In [ ]:
#EDSL has a cache directory that stores the responses from the LLM. Since we need new responses everytime, we need to clear the cache.    
import shutil
import os

# Path to the .edsl_cache directory
cache_dir = '.edsl_cache'

# Check if the directory exists
if os.path.exists(cache_dir):
    # Remove the directory and its contents
    shutil.rmtree(cache_dir)

In [ ]:
import random
import re
import pandas as pd
from edsl.questions import QuestionMultipleChoice
from edsl import Agent, Model, Cache, Survey

In [ ]:
#Enter the model you want to use here
model = Model('gpt-4-turbo')
#Set the rate limits for the model, lower tier OpenAI accounts have lower rate limits, therfore we need to explicitly set the rate limits for the model
model._set_rate_limits(rpm=10000, tpm=2000000)

In [ ]:
MOUNTAINS_DISTRIBUTION = ["low", "low", "low", "medium", "high"]  # Distribution of mountains
GEMS_MAPPING = {"low":"Topaz", "medium":"Ruby", "high":"Diamond"} #Mapping of the mountains to the gems

In [ ]:
no_of_players = 5 #Number of players in the experiment

In [ ]:
#Agent traits, note that we can create a large number of agents and use only a subset of them in the experiment. 
agent_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
ages = [22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
locations = ["New York", "California", "Texas", "Florida", "Washington", "Nevada", "Oregon", "Colorado", "Arizona", "Illinois"]
names = ["Jack", "Jane", "David", "Cobb", "Samantha", "Michael", "Sarah", "Daniel", "Emily", "Joshua"]

In [ ]:
#Switch for ambiguity and rivalry
ambiguity = False #Set to True if you want to run the experiment with ambiguity
rivalrous = False #Set to True if you want to run the experiment with rivalry   
#Dynamic sub-instruction generation based on the parameters
if ambiguity:
    location_of_gems = f"""Each mountain could hold a Diamond, a Ruby, or a Topaz. However, which mountains contain which gems is unknown, and nobody knows how many gems per type there are. At the beginning of each round, each type of gem is randomly assigned to a different mountain, so any gem could be hidden in any mountain."""
else:
    location_of_gems = f"""In each round, there are: {MOUNTAINS_DISTRIBUTION.count("low")} mountain(s) containing topazes; {MOUNTAINS_DISTRIBUTION.count("medium")} mountain(s) containing rubies; {MOUNTAINS_DISTRIBUTION.count("high")} mountain(s) containing diamonds. However, which mountains contain which gems is unknown. At the beginning of each round, each type of gem is randomly assigned to a different mountain, so any gem could be hidden in any mountain."""
    

Agent instructions are defined below, with the respective placeholders according to the parameters.

In [ ]:
non_rivalrous_instructions = f"""
General Information:
Welcome. This is an experiment in the economics of decision-making. If you pay close attention to these instructions, you can earn a significant amount of money paid to you at the end of the experiment. Following these instructions, you will be asked to make some choices. There are no correct choices. Your choices depend on your preferences and beliefs, so different participants will usually make different choices. You will be paid according to your choices, so read these instructions carefully and think before you decide.

The Basic Idea:
There are {len(MOUNTAINS_DISTRIBUTION)} mountains and each of them hides one type of gem, which can only be found by exploring the mountain. There are 3 types of gems hidden in the {len(MOUNTAINS_DISTRIBUTION)} mountains: Diamonds, Rubies, and Topazes. The exact values of the topazes, rubies, and diamonds vary across rounds but the diamonds are always worth more than the rubies and the rubies are always worth more than the topazes. You choose which mountains to explore and the value of the gems you find are your earnings in dollars. Your objective is to maximize your own earnings.

Location of Gems:
{location_of_gems}

How Participants Choose Mountains:
There are {no_of_players} participants including you in total. In each round, participants choose which mountain to explore. The choice does not happen simultaneously, but participants choose sequentially, one after the other, according to a random order. You can choose to explore any mountain you wish. If you choose the same mountain chosen by other participants, each of you will receive the full value uncovered. Similarly, if someone else chooses the same mountain that you previously chose, you will still receive the full gem's value (and so will the other participant(s) who chose it). This means that payoffs are non-rival and there is no penalty in choosing the same mountain as other players. To repeat, no participant has any private information in Stage 1 on the location of the gems.

Each Round Has 2 Stages:
A round consists of 2 stages. At the beginning of a new round, gems are randomly allocated to the {len(MOUNTAINS_DISTRIBUTION)} different mountains. The position of gems will not be reset between the two stages in a round. In Stage 1, all participants sequentially choose one mountain to explore. Before choosing a mountain, you will see which mountains have been selected by the other participants in your group who chose before you. You can choose the same mountain chosen by other participants or a different mountain. At the end of Stage 1, the gems hidden in each mountain selected by all participants in Stage 1 are revealed, and you earn the value of the gem hidden in the mountain you chose. In Stage 2, you can again choose any of the same {len(MOUNTAINS_DISTRIBUTION)} mountains; that is, you can either choose the same mountain from Stage 1 or switch to another one. The position of gems remains the same as in Stage 1, but this time you will also see the gems located in the mountains revealed in Stage 1. At the end of Stage 2, the gems hidden in each mountain selected by all participants in Stage 2 are revealed, and you earn the value of the gem hidden in the mountain you chose in Stage 2. Your total earnings for the round equal the sum of the value of the gem you found in Stage 1 and the value of the gem you found in Stage 2. Again, if multiple players choose the same mountain, they all receive its full value.

Payment:
At the end of the round, you will be paid an amount equivalent to the sum of payoffs you earned in Stage 1 and Stage 2. This protocol of determining payments suggests that you should choose in each Stage knowing that your choice directly determines your payment because the dollar value of the gems you select will directly translate into your earnings. 

Frequently Asked Questions:
Q1: Is this some kind of psychology experiment with an agenda you haven't told us?A: No, it is an economics experiment. These instructions are meant to clarify how you earn money and our interest is in seeing how people make decisions.
Q2: Is there a ''correct'' or ''wrong'' choice of action? Is this kind of a test?A: No, your optimal choice depends on your preferences and beliefs and different people may hold different beliefs.
Q3: Will there be any mountains with empty payoffs (no gem at all)? A: No, each and every mountain contains a gem and you are guaranteed a payoff for choosing any mountain whether it is a topaz, ruby, or a diamond.
Q4: Will there be any negative payoffs for some hidden mountains?A: No, there is no potentially lower payoff than of a topaz, which is always positive.
Q5: Are the payoffs split if more players choose the same mountain?A: No, each participant receives the full value of the gem uncovered, regardless of how many participants choose a mountain in that specific stage.

Instructions for the Risk Preference:
The Choice: You will be asked to choose between two options, "Option A" and "Option B" where:

"Option A" always pays $4.00 with probability p and $3.20 otherwise.
"Option B" always pays $7.70 with probability p and $0.20 otherwise.

Repeated Choices:

You will be asked to make a choice between "Option A" and "Option B" not once, but ten times where p will increase from 10% to 100%, 10% at a time.
For example, the first choice will have p=10% and you will choose whether you prefer "Option A" ($4.00 with a 10% chance or $3.20 otherwise) or "Option B" ($7.70 with a 10% chance or $0.20 otherwise).

Each successive choice will increase p by 10 percentage points until the last choice where "Option A" will pay $4.00 with certainty, and "Option B" will pay $7.70 with certainty.
Note: Once you switch from choosing "Option A" to "Option B", it makes sense that you will continue to choose "Option B" in all consecutive choice problems. For example, if you prefer "Option B" when p=80%, then it makes sense to prefer "Option B" when p=90% and when p=100%, since "Option B" is even more attractive in these choice problems.

Payment for risk preference task:
The computer will randomly select one of the 10 choice problems and pay you according to your choice in that problem where the computer will decide the outcome based on the value of p. 
"""


In [ ]:
rivalrous_instructions = f"""
General Information:
Welcome. This is an experiment in the economics of decision-making. If you pay close attention to these instructions, you can earn a significant amount of money paid to you at the end of the experiment. Following these instructions, you will be asked to make some choices. There are no correct choices. Your choices depend on your preferences and beliefs, so different participants will usually make different choices. You will be paid according to your choices, so read these instructions carefully and think before you decide.

The Basic Idea:
There are {len(MOUNTAINS_DISTRIBUTION)} mountains and each of them hides one type of gem, which can only be found by exploring the mountain. There are 3 types of gems hidden in the {len(MOUNTAINS_DISTRIBUTION)} mountains: Diamonds, Rubies, and Topazes. The exact values of the topazes, rubies, and diamonds vary across rounds but the diamonds are always worth more than the rubies and the rubies are always worth more than the topazes. You choose which mountains to explore and the value of the gems you find influence your earnings in dollars. Your objective is to maximize your own earnings.

Location of Gems:
{location_of_gems}

How Participants Choose Mountains:
There are {no_of_players} participants including you in total. In each round, participants choose which mountain to explore. The choice does not happen simultaneously, but participants choose sequentially, one after the other, according to a random order. You can choose to explore any mountain you wish. If you choose the same mountain chosen by other participants, each of you will equally split the value of the uncovered gem in that specific stage. Similarly, if someone else chooses the same mountain that you previously chose, you will still receive the split value of the gem (and so will the other participant(s) who chose it) in that specific stage. This means that payoffs are rival and there is some penalty in choosing the same mountain as other players. To repeat, no participant has any private information in Stage 1 on the location of the gems.

Each Round Has 2 Stages:
A round consists of 2 stages. At the beginning of a new round, gems are randomly allocated to the {len(MOUNTAINS_DISTRIBUTION)} different mountains. The position of gems will not be reset between the two stages in a round. In Stage 1, all participants sequentially choose one mountain to explore. Before choosing a mountain, you will see which mountains have been selected by the other participants in your group who chose before you. You can choose the same mountain chosen by other participants or a different mountain. At the end of Stage 1, the gems hidden in each mountain selected by all participants in Stage 1 are revealed, and you earn the payoffs based on the gem hidden in the mountain you chose and whether other players chose the same mountain or not in that specific stage. In Stage 2, you can again choose any of the same {len(MOUNTAINS_DISTRIBUTION)} mountains; that is, you can either choose the same mountain from Stage 1 or switch to another one. The position of gems remains the same as in Stage 1, but this time you will also see the gems located in the mountains revealed in Stage 1. At the end of Stage 2, the gems hidden in each mountain selected by all participants in Stage 2 are revealed, and you earn the split value of the gem hidden in the mountain you chose in Stage 2. Your total earnings for the round equal the sum of the split value of the gem you found in Stage 1 and the split value of the gem you found in Stage 2. Again, if multiple players choose the same mountain, they all equally split its full value in that specific stage.

Payment:
At the end of the round, you will be paid an amount equivalent to the sum of payoffs you earned in Stage 1 and Stage 2. This protocol of determining payments suggests that you should choose in each Stage knowing that your choice directly determines your payment because the dollar value of the gems you select will directly translate into your earnings. 

Frequently Asked Questions:
Q1: Is this some kind of psychology experiment with an agenda you haven't told us?A: No, it is an economics experiment. These instructions are meant to clarify how you earn money and our interest is in seeing how people make decisions.
Q2: Is there a ''correct'' or ''wrong'' choice of action? Is this kind of a test?A: No, your optimal choice depends on your preferences and beliefs and different people may hold different beliefs.
Q3: Will there be any mountains with empty payoffs (no gem at all)? A: No, each and every mountain contains a gem and you are guaranteed a payoff for choosing any mountain whether it is a topaz, ruby, or a diamond.
Q4: Will there be any negative payoffs for some hidden mountains?A: No, there is no potentially lower payoff than of a topaz, which is always positive.
Q5: Are the payoffs split if more players choose the same mountain?A: Yes, the payoffs are split depending on how many participants choose a mountain in that specific stage. For instance, if 3 agents choose a mountain of value $12 in stage 1, they all get $4 each for that stage. Similarly, if 2 agents choose a mountain with a payoff of $12 in stage 2, they get $6 each for stage 2.

Instructions for the Risk Preference:
The Choice: You will be asked to choose between two options, "Option A" and "Option B" where:

"Option A" always pays $4.00 with probability p and $3.20 otherwise.
"Option B" always pays $7.70 with probability p and $0.20 otherwise.

Repeated Choices:

You will be asked to make a choice between "Option A" and "Option B" not once, but ten times where p will increase from 10% to 100%, 10% at a time.
For example, the first choice will have p=10% and you will choose whether you prefer "Option A" ($4.00 with a 10% chance or $3.20 otherwise) or "Option B" ($7.70 with a 10% chance or $0.20 otherwise).

Each successive choice will increase p by 10 percentage points until the last choice where "Option A" will pay $4.00 with certainty, and "Option B" will pay $7.70 with certainty.
Note: Once you switch from choosing "Option A" to "Option B", it makes sense that you will continue to choose "Option B" in all consecutive choice problems. For example, if you prefer "Option B" when p=80%, then it makes sense to prefer "Option B" when p=90% and when p=100%, since "Option B" is even more attractive in these choice problems.

Payment for risk preference task:
The computer will randomly select one of the 10 choice problems and pay you according to your choice in that problem where the computer will decide the outcome based on the value of p. 
"""

In [ ]:
if rivalrous:
    instructions = rivalrous_instructions
else:
    instructions = non_rivalrous_instructions

In [ ]:

#Create agents with the respective traits and instructions
agents = [Agent(
        name=name, 
        traits={"age": age, "location": location, "agent_id": agent_id},
        instruction=instructions) for name, age, location, agent_id in zip(names, ages, locations, agent_ids)]

Pre-experiment quiz to check the understanding of the instructions:

In [ ]:
q1 = QuestionMultipleChoice(
   question_name = "same_mountain",
   question_text = "In each round, you will select two mountains (one in Stage 1, and one in Stage 2) and collect the gem that they hide. You can choose the same mountain in both stages, or change after Stage 1?",
   question_options = ["Correct", "Incorrect"]
)
q2 = QuestionMultipleChoice(
   question_name = "value_split",
   question_text = "If more than one player selects the same mountain, the value of the gem will be split among all the participants who chose it and there is a competition among the players?",
   question_options = ["Correct", "Incorrect"]
)
q3 = QuestionMultipleChoice(
   question_name = "gems_shuffling",
   question_text = "At the beginning of a new round, the gems are reshuffled and randomly allocated to a different mountain?",
   question_options = ["Correct", "Incorrect"]
)
q4 = QuestionMultipleChoice(
   question_name = "private_information",
   question_text = "No group member has any private initial information in Stage 1 on the location of gems?",
   question_options = ["Correct", "Incorrect"]
)
q5 = QuestionMultipleChoice(
   question_name = "stage_2_reset",
   question_text = "The position of gems will not be reset between the two stages of a round?",
   question_options = ["Correct", "Incorrect"]
)
q6 = QuestionMultipleChoice(
   question_name = "simultaneous_or_not",
   question_text = "All group members choose their mountain simultaneously?",
   question_options = ["Correct", "Incorrect"]
)
q7 = QuestionMultipleChoice(
   question_name = "group_member_before_you",
   question_text = "If another group member chose a mountain before you, you cannot choose it again?",
   question_options = ["Correct", "Incorrect"]
)
q8 = QuestionMultipleChoice(
   question_name = "round_payment_random",
   question_text = "Each and every mountain has a gem and no mountain is empty at all. Also a topaz gem has the lowest payoff and there is nothing worse than that?",
   question_options = ["Correct", "Incorrect"]
)

In [ ]:

quiz = Survey(questions = [q1, q2, q3, q4, q5, q6, q7, q8])
quiz = quiz.set_full_memory_mode()
c = Cache() # create empty Cache object

quiz_results = quiz.by(agents).by(model).run(cache = c)


In [ ]:
#Correct answers for the quiz based on the parameters
if rivalrous:
    correct_answers = ['Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Incorrect', 'Incorrect', 'Correct']
else:
    correct_answers = ['Correct', 'Incorrect', 'Correct', 'Correct', 'Correct', 'Incorrect', 'Incorrect', 'Correct']


In [ ]:

agent_names = []
question_one_answer = []
question_one_comment = []
question_two_answer = []
question_two_comment = []
question_three_answer = []
question_three_comment = []
question_four_answer = []
question_four_comment = []
question_five_answer = []
question_five_comment = []
question_six_answer = []
question_six_comment = []
question_seven_answer = []
question_seven_comment = []
question_eight_answer = []
question_eight_comment = []

# Gather quiz results
for item in quiz_results:
    agent_names.append(item['agent']['name'])
    question_one_answer.append(item['answer']['same_mountain'])
    question_one_comment.append(item['answer']['same_mountain_comment'])
    question_two_answer.append(item['answer']['value_split'])
    question_two_comment.append(item['answer']['value_split_comment'])
    question_three_answer.append(item['answer']['gems_shuffling'])
    question_three_comment.append(item['answer']['gems_shuffling_comment'])
    question_four_answer.append(item['answer']['private_information'])
    question_four_comment.append(item['answer']['private_information_comment'])
    question_five_answer.append(item['answer']['stage_2_reset'])
    question_five_comment.append(item['answer']['stage_2_reset_comment'])
    question_six_answer.append(item['answer']['simultaneous_or_not'])
    question_six_comment.append(item['answer']['simultaneous_or_not_comment'])
    question_seven_answer.append(item['answer']['group_member_before_you'])
    question_seven_comment.append(item['answer']['group_member_before_you_comment'])
    question_eight_answer.append(item['answer']['round_payment_random'])
    question_eight_comment.append(item['answer']['round_payment_random_comment'])

# Create a DataFrame
df = pd.DataFrame({
    'agent_names': agent_names,
    'question_1_answer': question_one_answer,
    'question_1_comment': question_one_comment,
    'question_2_answer': question_two_answer,
    'question_2_comment': question_two_comment,
    'question_3_answer': question_three_answer,
    'question_3_comment': question_three_comment,
    'question_4_answer': question_four_answer,
    'question_4_comment': question_four_comment,
    'question_5_answer': question_five_answer,
    'question_5_comment': question_five_comment,
    'question_6_answer': question_six_answer,
    'question_6_comment': question_six_comment,
    'question_7_answer': question_seven_answer,
    'question_7_comment': question_seven_comment,
    'question_8_answer': question_eight_answer,
    'question_8_comment': question_eight_comment
})

# Function to calculate scores
def calculate_score(row):
    return sum([row[f'question_{i}_answer'] == correct_answers[i-1] for i in range(1, 9)])

# Apply function to calculate scores
df['score'] = df.apply(calculate_score, axis=1)

# Save DataFrame to CSV
df.to_csv('agent_scores_no-data.csv', index=False)

# Update agent traits with quiz results
for i, item in enumerate(quiz_results):
    text = "Quiz results:\n"
    questions_answers_comments = [
        (1, question_one_answer[i], question_one_comment[i], correct_answers[0]),
        (2, question_two_answer[i], question_two_comment[i], correct_answers[1]),
        (3, question_three_answer[i], question_three_comment[i], correct_answers[2]),
        (4, question_four_answer[i], question_four_comment[i], correct_answers[3]),
        (5, question_five_answer[i], question_five_comment[i], correct_answers[4]),
        (6, question_six_answer[i], question_six_comment[i], correct_answers[5]),
        (7, question_seven_answer[i], question_seven_comment[i], correct_answers[6]),
        (8, question_eight_answer[i], question_eight_comment[i], correct_answers[7]),
    ]
    for q_num, answer, comment, correct in questions_answers_comments:
        text += f"Question Number {q_num}:\n"
        text += f"Question: {df.columns[df.columns.get_loc(f'question_{q_num}_answer') - 1]}\n"
        text += f"Your Answer: {answer}\n"
        text += f"Correct Answer: {correct}\n"
        text += f"Your rationale: {comment}\n"
    agents[i]['traits']['quiz'] = text


In [ ]:
def create_array(text, text_count, total_length):
    if text_count > total_length:
        raise ValueError("The text count cannot be greater than the total length of the array.")
    
    # Create an array with the specified number of text entries
    array = [text] * text_count
    
    # Add 'None' to fill the rest of the array
    array.extend(['None'] * (total_length - text_count))
    
    return array

In [ ]:
# risk_priming = ['This agent enjoys taking high risks with the potential for high rewards. This means the agent will go for high payoff choices even though they might end up with low outcomes.', 'This agent enjoys taking high risks with the potential for high rewards. This means the agent will go for high payoff choices even though they might end up with low outcomes.', 'This agent enjoys taking high risks with the potential for high rewards. This means the agent will go for high payoff choices even though they might end up with low outcomes.', 'This agent balances risk and reward, considering both potential gains and potential losses. The agent is likely to choose moderate-risk options that offer a reasonable chance of a good outcome without extreme potential for loss. They weigh the probabilities and impacts of different choices carefully, aiming for a balance between safety and opportunity. ', 'This agent balances risk and reward, considering both potential gains and potential losses. The agent is likely to choose moderate-risk options that offer a reasonable chance of a good outcome without extreme potential for loss. They weigh the probabilities and impacts of different choices carefully, aiming for a balance between safety and opportunity. ', 'This agent balances risk and reward, considering both potential gains and potential losses. The agent is likely to choose moderate-risk options that offer a reasonable chance of a good outcome without extreme potential for loss. They weigh the probabilities and impacts of different choices carefully, aiming for a balance between safety and opportunity.', 'This agent prefers to avoid uncertainty and potential losses. This means the agent will choose safer, low-risk options even if the potential rewards are smaller. ', 'This agent prefers to avoid uncertainty and potential losses. This means the agent will choose safer, low-risk options even if the potential rewards are smaller. ', 'This agent prefers to avoid uncertainty and potential losses. This means the agent will choose safer, low-risk options even if the potential rewards are smaller. ','This agent prefers to avoid uncertainty and potential losses. This means the agent will choose safer, low-risk options even if the potential rewards are smaller. ']
risk_priming = ["None","None","None","None","None","None","None","None","None","None"]
# risk_text = """This agent is risk-loving. If the diamond has not been found yet, they will pick an unchosen mountain to give them the chance of discovering one, since this is the kind of risk they like to take."""
# text_count = 1
# total_length = 10

# risk_priming = create_array(risk_text, text_count, total_length)


for i, agent in enumerate(agents):
    agent['traits']['risk_priming'] = risk_priming[i]

In [ ]:
prosociality = ["None","None","None","None","None","None","None","None","None","None"]


# prosocial_text = """This agent is pro-social and aims to help the other players. This means they prefer unchosen mountains to give the other members of the group the chance of discovering a diamond."""
# text_count = 5
# total_length = 10
# prosociality = create_array(prosocial_text, text_count, total_length)

for i, agent in enumerate(agents):
    agent['traits']['prosociality'] = prosociality[i]

In [ ]:
exploration_priming = ["None","None","None","None","None","None","None","None","None","None"]

# exploration_text = """This agent only wants to find the diamond. They will keep exploring until they find the diamond, since this is their ultimate goal."""
# text_count = 5
# total_length = 10
# exploration_priming = create_array(exploration_text, text_count, total_length)

for i, agent in enumerate(agents):
    agent['traits']['exploration_priming'] = exploration_priming[i]

#Risk preference survey
#10 questions, 10% increments from 10% to 100%


In [ ]:
q1_risk = QuestionMultipleChoice(
    question_name = "risk_question_1",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 10%, $3.20 otherwise, Option B - $7.70 with a chance of 10%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q2_risk = QuestionMultipleChoice(
    question_name = "risk_question_2",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 20%, $3.20 otherwise, Option B - $7.70 with a chance of 20%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q3_risk = QuestionMultipleChoice(
    question_name = "risk_question_3",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 30%, $3.20 otherwise, Option B - $7.70 with a chance of 30%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q4_risk = QuestionMultipleChoice(
    question_name = "risk_question_4",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 40%, $3.20 otherwise, Option B - $7.70 with a chance of 40%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q5_risk = QuestionMultipleChoice(
    question_name = "risk_question_5",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 50%, $3.20 otherwise, Option B - $7.70 with a chance of 50%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q6_risk = QuestionMultipleChoice(
    question_name = "risk_question_6",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 60%, $3.20 otherwise, Option B - $7.70 with a chance of 60%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q7_risk = QuestionMultipleChoice(
    question_name = "risk_question_7",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 70%, $3.20 otherwise, Option B - $7.70 with a chance of 70%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q8_risk = QuestionMultipleChoice(
    question_name = "risk_question_8",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 80%, $3.20 otherwise, Option B - $7.70 with a chance of 80%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q9_risk = QuestionMultipleChoice(
    question_name = "risk_question_9",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 90%, $3.20 otherwise, Option B - $7.70 with a chance of 90%, $0.20 otherwise",
    question_options = ["A", "B"]
)

q10_risk = QuestionMultipleChoice(
    question_name = "risk_question_10",
    question_text = "Which option do you choose: Option A - $4.00 with a chance of 100%, $3.20 otherwise, Option B - $7.70 with a chance of 100%, $0.20 otherwise",
    question_options = ["A", "B"]
)


In [ ]:
risk_preference = Survey(questions = [q1_risk, q2_risk, q3_risk, q4_risk, q5_risk, q6_risk, q7_risk, q8_risk, q9_risk, q10_risk])
risk_preference = risk_preference.set_full_memory_mode()
c = Cache() # create empty Cache object

risk_preference_results = risk_preference.by(agents).by(Model('gpt-4-turbo')).run(cache = c)

In [ ]:
questions = [
    q1_risk, q2_risk, q3_risk, q4_risk, q5_risk, q6_risk, q7_risk, q8_risk, q9_risk, q10_risk
]

# Initialize lists
agent_names = []
risk_question_answers = [[] for _ in range(10)]
risk_question_comments = [[] for _ in range(10)]

# Iterate over the risk preference results
for item in risk_preference_results:
    agent_names.append(item['agent']['name'])
    for i in range(len(questions)):
        risk_question_answers[i].append(item['answer'][f'risk_question_{i+1}'])
        risk_question_comments[i].append(item['answer'][f'risk_question_{i+1}_comment'])
    
    # Create a text format combining all the questions, agent's answers, and correct answers
    text = "Risk Preferences results:\n"
    for i in range(len(questions)):
        question_text = questions[i]['question_text']
        answer = risk_question_answers[i][-1]
        comment = risk_question_comments[i][-1]
        text += f"Question Number {i+1}:\nQuestion: {question_text}\nYour Answer: {answer}\nYour Rationale: {comment}\n"
    
    # Add the text to the agent's traits
    item['agent']['traits']['risk_survey'] = text

# Create a DataFrame
data = {
    'agent_names': agent_names
}

for i in range(len(questions)):
    data[f'risk_question_{i+1}_answer'] = risk_question_answers[i]
    data[f'risk_question_{i+1}_comment'] = risk_question_comments[i]

df = pd.DataFrame(data)

# Save DataFrame to CSV
df.to_csv('risk_preferences_no-data.csv', index=False)


In [ ]:
#checking the traits of the agents
agents[1]['traits']

In [ ]:
def agent_response(agent_id, context, agents):
    
    c = Cache() # create empty Cache object
    question_options = ["Mountain 1", "Mountain 2", "Mountain 3", "Mountain 4", "Mountain 5"]
    random.shuffle(question_options)
    q = QuestionMultipleChoice(
        question_name = "select_mountain",
        question_text = f"You are being asked to choose a mountain. Context: {context}\n Which mountain do you choose?",
        question_options = question_options
    )

    try:
        result = q.by(agents[agent_id]).by(model).run(cache = c)
        result.show_exceptions(traceback=True)
        agent_choice = result.select("answer.select_mountain").first()
        agent_comment = result.select("comment.select_mountain_comment").first()
    except:
        agent_choice = None
        agent_comment = "Picked a choice completely random, without any rational due to an error."

    # To show the prompts that were used:
    # result.select("agent_name", "prompt.select_mountain_system_prompt").print()
    # result.select("agent_name", "prompt.select_mountain_system_prompt", "prompt.select_mountain_user_prompt", "answer.select_mountain", "comment.select_mountain_comment").print(format="rich")

    # Return just the response:
    return agent_choice, agent_comment

In [ ]:
# Load existing master file or create a new one if it doesn't exist
try:
    master_data = pd.read_csv("master_data_points_no-data.csv")
    print("Loaded existing master data file.")
except FileNotFoundError:
    master_data = pd.DataFrame(columns=["round_number","condition","risk_priming","ambiguity","rivalrous","revealed_mountain","sub_round_number","stage_number","agent_id", "choice", "choice_order", "comment_by_the_agent", "agent_payoff", "total_payoff_of_the_mountain", "payoff_class", "mountains_arrangement","mountains_payoffs", "prosociality", "exploration_priming", "error_flag"])
    print("Created new master data file.")

# Your existing data_points DataFrame
data_points = pd.DataFrame(columns=["round_number","condition","risk_priming","ambiguity","rivalrous","revealed_mountain","sub_round_number","stage_number","agent_id", "choice", "choice_order", "comment_by_the_agent", "agent_payoff", "total_payoff_of_the_mountain", "payoff_class", "mountains_arrangement","mountains_payoffs", "prosociality", "exploration_priming", "error_flag"])

In [ ]:
# Function to simulate the game
def simulate_game(overall_round_number,sub_round_number,mountain_values,reveal_type, agent_list):
    mountains = random.sample(MOUNTAINS_DISTRIBUTION, len(MOUNTAINS_DISTRIBUTION))
    revealed_value = None
    agents_context_p1={}
    agents_context_p2={}
    mountain_map = {}  # Dictionary to store mountain index to value mappings
    inverse_mountains_payoffs = {v: k for k, v in mountain_values.items()} # Invert the dictionary so values become keys

    # Reveal data if applicable
    if reveal_type in mountain_values:
        for mountain in mountains:
            if mountain_values[mountain] == mountain_values[reveal_type]:
                # revealed_value = (mountains.index(mountain), mountain_values[mountain])
                revealed_value = (mountains.index(mountain), GEMS_MAPPING[reveal_type])
                mountain_map["Mountain "+str(mountains.index(mountain)+1)] = mountain_values[mountain]  # Store revealed mountain mapping
                break

    print(f"Mountains (hidden for the agents): {mountains}")

    if ambiguity:
        context = "Mountains are hidden in a random order. Payoffs are hidden in mountains in a random order and all the mountains have a gem of a certain value, with  Diamond > Ruby > Topaz."
    else:
        context = f""" This round has three payoffs of value ${mountain_values['low']} (Topazes), 
       one value of ${mountain_values['medium']} (Ruby) and one value of ${mountain_values['high']} (Diamond).
        Mountains are hidden in a random order. Payoffs are hidden in mountains in a random order and all the mountains have a gem of a certain value, with  Diamond > Ruby > Topaz. 
        """
    if revealed_value:
        # print(f"Revealed mountain: Mountain {revealed_value[0]+1} with value ${revealed_value[1]}")
        # context = context + f"Revealed Mountain: Mountain {revealed_value[0]+1} with value ${revealed_value[1]} \n"
        print(f"Revealed mountain: Mountain {revealed_value[0]+1} has a gem {revealed_value[1]}")
        context = context + f"Revealed Mountain: Mountain {revealed_value[0]+1} has a gem {revealed_value[1]} \n"
    else:
        print("No mountains are revealed for this round")
        context = context + "No mountains are revealed for this round \n"
    
    agent_order = [x - 1 for x in agent_list]  # List of agent indices
    random.shuffle(agent_order)   # Shuffle the list

    # Agents make choices for stage 1
    choices = []
    print("Agents are making their choices for Stage 1...\n")
    context = context + "Agents are making their choices for Stage 1... \n"
    for j,i in enumerate(agent_order): 
        agents_context_p1[i+1] = context
        choice, comment_by_the_agent = agent_response(i,context, agents)
        if choice:
            choice = str(choice)
            error_flag = 0
        else:
            choice = random.choice(["Mountain 1", "Mountain 2", "Mountain 3", "Mountain 4", "Mountain 5"])
            choice = str(choice)
            error_flag = 1
        choices.append(choice)
        mountain_index = int(re.findall('\d+\.\d+|\d+', choice)[0]) - 1
        mountain_map[choice] = mountain_values[mountains[mountain_index]]  # Update mountain mapping with actual value
        temp = f"Agent {i+1}, enter your choice of mountain: {choice}"
        context =  context + f"An Agent chose {choice}" + "\n" 
        print(temp)
        payoff_value = mountain_values[mountains[int(re.findall('\d+\.\d+|\d+', choice)[0]) - 1]]
        data_points.loc[len(data_points)] = [overall_round_number,reveal_type, agents[i]['traits']['risk_priming'],ambiguity, rivalrous, "None", sub_round_number, 1, i+1, choice, j+1, comment_by_the_agent, 0, payoff_value, inverse_mountains_payoffs.get(payoff_value), mountains, mountain_values, agents[i]['traits']['prosociality'], agents[i]['traits']['exploration_priming'], error_flag]
        master_data.loc[len(data_points)] = [overall_round_number,reveal_type, agents[i]['traits']['risk_priming'],ambiguity, rivalrous, "None", sub_round_number, 1, i+1, choice, j+1, comment_by_the_agent, 0, payoff_value, inverse_mountains_payoffs.get(payoff_value), mountains, mountain_values, agents[i]['traits']['prosociality'], agents[i]['traits']['exploration_priming'], error_flag]


    
    print(f"\nStage 1 Choices (by position) of peers: {[x for x in choices]}")
    
    if rivalrous:
        # Share payoffs if multiple agents choose the same mountain
        choice_counts = {choice: choices.count(choice) for choice in set(choices)}
        outcomes = [mountain_values[mountains[int(re.findall('\d+\.\d+|\d+', choice)[0]) - 1]] / choice_counts[choice] for choice in choices]
    else:
        outcomes = [mountain_values[mountains[int(re.findall('\d+\.\d+|\d+', choice)[0]) - 1]] for choice in choices]

    # Update outcomes in data_points for Stage 1
    for idx, (choice, outcome) in enumerate(zip(choices, outcomes)):
        data_points.loc[(data_points['round_number'] == overall_round_number) &
                        (data_points['sub_round_number'] == sub_round_number) &
                        (data_points['stage_number'] == 1) &
                        (data_points['agent_id'] == agent_order[idx] + 1), 'agent_payoff'] = outcome
    print("Learnings from Stage 1:")
    context = context + "Learnings from Stage 1:\n"

    for index, value in mountain_map.items():
        print(f"{index} is revealed to have a value of ${value}")
        context = context + f"{index} is revealed to have a value of ${value} \n"


    # Agents make choices for stage 2
    print("\nAgents are making their choices for Stage 2...")
    context = context + "Agents are making their choices for Stage 2... \n"

    new_choices = []
    for j,i in enumerate(agent_order):
        agents_context_p2[i+1] = context
        choice, comment_by_the_agent = agent_response(i,context, agents)
        if choice:
            choice = str(choice)
            error_flag = 0
        else:
            choice = random.choice(["Mountain 1", "Mountain 2", "Mountain 3", "Mountain 4", "Mountain 5"])
            choice = str(choice)
            error_flag = 1
        new_choices.append(choice)
        temp = f"Agent {i+1}, enter your choice of mountain: {choice}"
        context = context + f"An Agent chose {choice}\n"
        print(temp)
        payoff_value = mountain_values[mountains[int(re.findall('\d+\.\d+|\d+', choice)[0]) - 1]]
        data_points.loc[len(data_points)] = [overall_round_number, reveal_type, agents[i]['traits']['risk_priming'],ambiguity, rivalrous, "None", sub_round_number, 2, i+1, choice, j+1, comment_by_the_agent, 0, payoff_value, inverse_mountains_payoffs.get(payoff_value), mountains,mountain_values, agents[i]['traits']['prosociality'], agents[i]['traits']['exploration_priming'], error_flag]
        master_data.loc[len(data_points)] = [overall_round_number, reveal_type, agents[i]['traits']['risk_priming'],ambiguity, rivalrous, "None", sub_round_number, 2, i+1, choice, j+1, comment_by_the_agent, 0, payoff_value, inverse_mountains_payoffs.get(payoff_value), mountains,mountain_values, agents[i]['traits']['prosociality'], agents[i]['traits']['exploration_priming'], error_flag]

    
    print(f"\nStage 2 Choices: {[x for x in new_choices]}")

    if rivalrous:
        # Share payoffs if multiple agents choose the same mountain
        new_choice_counts = {choice: new_choices.count(choice) for choice in set(new_choices)}
        new_outcomes = [mountain_values[mountains[int(re.findall('\d+\.\d+|\d+', choice)[0]) - 1]] / new_choice_counts[choice] for choice in new_choices]
    else:
        new_outcomes = [mountain_values[mountains[int(re.findall('\d+\.\d+|\d+', choice)[0]) - 1]] for choice in new_choices]

    # Update outcomes in data_points for Stage 2
    for idx, (choice, outcome) in enumerate(zip(new_choices, new_outcomes)):
        data_points.loc[(data_points['round_number'] == overall_round_number) &
                        (data_points['sub_round_number'] == sub_round_number) &
                        (data_points['stage_number'] == 2) &
                        (data_points['agent_id'] == agent_order[idx] + 1), 'agent_payoff'] = outcome 

    print(f"Stage 2 Outcomes: {new_outcomes}")
    
    # Calculate total payoffs
    total_payoffs = [x + y for x, y in zip(outcomes, new_outcomes)]
    payoffs_dict = {agent: payoff for agent, payoff in zip(agent_order, total_payoffs)}

    for i in agent_order:
        print(f"\nTotal payoff for Agent {i+1}: ${payoffs_dict[i]}")
        print(f"Their Stage 1 context: {agents_context_p1[i+1]}\nTheir Stage 2 context: {agents_context_p2[i+1]}")
    
    # After each game, append new data to master file
    master_data.to_csv("master_data_points_no-data.csv", index=False)
    print(f"Updated master data file. Total rows: {len(master_data)}")

    # Your existing code to save individual round data
    data_points.to_csv(f"v1_no-data_points-data_Seq.csv", index=False)
    
    return None

In [ ]:
mountain_values_list = [{'low': 1, 'medium': 6, 'high': 10}, {'low': 1, 'medium': 6, 'high': 10.5}, {'low': 1, 'medium': 7, 'high': 12}, {'low': 2, 'medium': 7, 'high': 11}, {'low': 3, 'medium': 8, 'high': 12}]

In [ ]:
# mountain_values_list = [{'low': 1, 'medium': 5, 'high': 10}, {'low': 1, 'medium': 5, 'high': 10.5}, {'low': 1, 'medium': 6, 'high': 12}, {'low': 2, 'medium': 6, 'high': 11}, {'low': 3, 'medium': 7, 'high': 12}]

In [ ]:
rounds = 20
overall_round_number = 1

In [ ]:
# # Function to shuffle and select agents, if you want to run the experiment with a different number of agents, you can modify the following function
# def get_shuffled_agents(agent_ids, num_agents, num_groups):
#     agents = []
#     for _ in range(num_groups):
#         random.shuffle(agent_ids)
#         for i in range(0, len(agent_ids), num_agents):
#             agents.append(agent_ids[i:i + num_agents])
#     return agents

# # Generate the list of agents for each of the 5 groups
# selected_agents_list = get_shuffled_agents(agent_ids, 5, 5)

In [ ]:
# Iterate through mountain values and game types
for mountain_values in mountain_values_list:
    for game_type in ('no-data',):  
        for group in range(1):  # For each group of agents
            # agent_list = selected_agents_list[group]  # Select the current group of agents
            agent_list = [1,2,3,4,5]
            for i in range(rounds):  # Each group plays 5 games
                print(f"Round {overall_round_number} and sub-round {i+1} of category {game_type} is going on")
                simulate_game(overall_round_number, i+1, mountain_values, reveal_type=game_type, agent_list=agent_list)
                overall_round_number += 1

In [ ]:
# for mountain_values in mountain_values_list:
#     for game_type in ('no_data'):
#         for i in range(rounds):
#             print(f"Round {overall_round_number} and sub-round {i+1} of category {game_type} is going on")
#             simulate_game(overall_round_number, i+1,mountain_values,reveal_type=game_type, agent_list=agent_list)
#             overall_round_number = overall_round_number + 1